# Project 1 Part 4 - Creating a master parcel database

In this part of the project, we will use Python to read, process, and double all of the parcel data into a database.  Note that this is not our only alternative, and in Project 1 Part 4 b, we will look at another alternative, that is reading all the of original, raw files into their own database table, then using SQL to join/link/aggregate the tables.

## Chunking Files in Pandas – Part 1 (20 Points)

In this part of the project, you will use `Panda`’s to process the data from the MinneMUDAC 2016 competition Dive into Water Data.  The data can be found at the [MinneMUDAC site](http://minneanalytics.org/minnemudac/data/).  You should document your work in a Jupyter notebook, which will be used to submit your solution.  **For the rest of the parts of this project, we will limit ourselves to the years 2004-2014.**

1. Remind me why we want to skip 2003.

2003 has a lot less columns in common with the other years

2. Import the common columns list and translation dictionaries from the `.py` file you created in the last part of the project.

In [1]:
from project_data_Buske import common_columns

In [2]:
from project_data_Buske import latlong_to_code

In [3]:
from project_data_Buske import latlong_to_name

In [4]:
from project_data_Buske import code_to_name

In [5]:
from project_data_Buske import code_to_distance

In [6]:
from project_data_Buske import latlong_to_distance

3. Use glob and a list comprehension to get a list of file names for the years 2004-2014.

In [7]:
from glob import glob
files = glob('./data/MinneMUDAC_raw_files/20**_metro_tax_parcels.txt')[2:-1]
[file for file in files]

['./data/MinneMUDAC_raw_files/2004_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/2005_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/2006_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/2007_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/2008_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/2009_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/2010_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/2011_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/2012_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/2013_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/2014_metro_tax_parcels.txt']

In [8]:
#parc = pd.read_csv('./data/MinneMUDAC_raw_files/2004_metro_tax_parcels.txt',sep = '|', dtype={'centroid_lat':str, 'centroid_long':str})

In [9]:
#parc.columns.shape

In [10]:
#list((sorted(common_columns)))

4. Use the first chunk of the first file to prototype an expression that <br>
    a. Selects the common columns <br>
    b. Fixes any issues with the column names <br>
    c. Changes columns to the correct types (if necessary).  More information about the columns can be found [here](ftp://ftp.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_metrogis/plan_regonal_prcls_open/metadata/metadata.html). It is **imperative** that you keep the lat and long columns as strings. <br>
    d. Use the translation dictionaries from the last part to add three new columns to the chunk: lake code, lake name, parcel distance to the lake.
    e. Filters to only properties that are within 1600 m (~1 mile) of the closest lake.

In [8]:
import pandas as pd
fileiterator = [pd.read_csv(file, chunksize = 50000, sep = '|', dtype={'centroid_lat': str, 'centroid_long': str}) for file in files]
fileiterator

In [12]:
for i, f in enumerate(fileiterator):
    print()

In [12]:
from toolz import first

In [13]:
firstchunk = first(fileiterator[0])
firstchunk.head()

/Users/tr8934qw/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2901: DtypeWarning: Columns (38,67) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


,ACRES_DEED,ACRES_POLY,AGPRE_ENRD,AGPRE_EXPD,AG_PRESERV,BASEMENT,BLDG_NUM,BLOCK,CITY,CITY_USPS,...,XUSE1_DESC,XUSE2_DESC,XUSE3_DESC,XUSE4_DESC,YEAR_BUILT,Year,ZIP,ZIP4,centroid_lat,centroid_long
0,0.0,8.03,NaN,NaN,N,NaN,NaN,NaN,SAINT FRANCIS,NaN,...,NaN,NaN,NaN,NaN,1980.0,2004,NaN,NaN,45.41332,-93.26739
1,0.0,0.93,NaN,NaN,N,NaN,24457.0,NaN,SAINT FRANCIS,BETHEL,...,NaN,NaN,NaN,NaN,1974.0,2004,55005,NaN,45.41354,-93.2701
2,0.0,8.75,NaN,NaN,N,NaN,24442.0,NaN,SAINT FRANCIS,BETHEL,...,NaN,NaN,NaN,NaN,1969.0,2004,55005,NaN,45.41318,-93.27344
3,0.0,11.17,NaN,NaN,N,NaN,410.0,NaN,SAINT FRANCIS,BETHEL,...,NaN,NaN,NaN,NaN,1989.0,2004,55005,NaN,45.41167,-93.27684
4,0.0,14.46,NaN,NaN,N,NaN,480.0,NaN,SAINT FRANCIS,BETHEL,...,NaN,NaN,NaN,NaN,1995.0,2004,55070,NaN,45.41169,-93.27849


In [14]:
from dfply import *
from more_dfply import *

In [15]:
cols_to_keep = ['ACRES_DEED',
                'ACRES_POLY',
                'AGPRE_ENRD',
                'AG_PRESERV',
                'BASEMENT',
                'CITY',
                'COOLING',
                'DWELL_TYPE',
                'EMV_BLDG',
                'EMV_LAND',
                'FIN_SQ_FT',
                'GARAGE', 
                'GARAGESQFT', 
                'GREEN_ACRE',
                'HOMESTEAD',
                #'ID',
                'LANDMARK',
                'OWN_ADD_L1',
                'OWN_ADD_L2',
                'OWN_ADD_L3',
                'PARC_CODE',
                'PIN', 
                'SALE_VALUE', 
                'SPEC_ASSES',
                'TAX_CAPAC', 
                'TAX_EXEMPT', 
                'TOTAL_TAX', 
                'USE1_DESC',
                'USE2_DESC',
                'USE3_DESC',
                'USE4_DESC',
                'WSHD_DIST',
                'XUSE1_DESC',
                'XUSE2_DESC',
                'XUSE3_DESC',
                'XUSE4_DESC',
                'YEAR_BUILT',
                'Year',
                'centroid_lat',
                'centroid_long']

In [69]:
exp = (firstchunk 
         >> select(cols_to_keep)
         >> mutate(lat_long = pd.Series(zip(firstchunk.centroid_lat, firstchunk.centroid_long)).astype(str))
         >> mutate(lake_code = recode(X.lat_long, latlong_to_code).astype(str))
         >> mutate(lake_name = recode(X.lat_long, latlong_to_name).astype(str))
         >> mutate(parcel_dist_to_lake = recode(X.lat_long, latlong_to_distance).astype(str))
         >> drop(X.lat_long)
         >> filter_by(~X.lake_name.isna())
         >> filter_by(~X.parcel_dist_to_lake.astype(float).isna())
         >> filter_by(X.parcel_dist_to_lake.astype(float) <= 1600)
      )         
exp

,ACRES_DEED,ACRES_POLY,AGPRE_ENRD,AG_PRESERV,BASEMENT,CITY,COOLING,DWELL_TYPE,EMV_BLDG,EMV_LAND,...,XUSE2_DESC,XUSE3_DESC,XUSE4_DESC,YEAR_BUILT,Year,centroid_lat,centroid_long,lake_code,lake_name,parcel_dist_to_lake


In [17]:
cols_to_drop = ['AGPRE_EXPD',#drop
                'BLDG_NUM',#drop
                'BLOCK',#drop
                'CITY_USPS',#drop
                'COUNTY_ID',#drop
                'EMV_TOTAL',#drop
                'HEATING', #drop
                'HOME_STYLE',#drop
                'LOT',#drop
                'MULTI_USES',#drop
                'NUM_UNITS',#drop
                'OPEN_SPACE',#drop
                'OWNER_MORE',#drop
                'OWNER_NAME',#drop
                'PLAT_NAME', #drop
                'PREFIXTYPE', #drop
                'PREFIX_DIR', #drop
                'SALE_DATE', #drop
                'SCHOOL_DST', #drop
                'STREETNAME', #drop
                'STREETTYPE', #drop
                'SUFFIX_DIR', #drop
                'Shape_Area', #drop
                'Shape_Leng', #drop
                'TAX_ADD_L1', #drop
                'TAX_ADD_L2', #drop
                'TAX_ADD_L3', #drop
                'TAX_NAME', #drop
                'UNIT_INFO', #drop
                'ZIP',#drop
                'ZIP4',#drop
                ]

5. Now convert your expression from the last problem to a function and test that this function works on the first few chunks of each file.

In [18]:
from functoolz import pipeable
read_parcel = lambda path: pd.read_csv(path, chunksize = 500, sep = '|', dtype={'centroid_lat':str, 'centroid_long':str})
read_parcel('./data/MinneMUDAC_raw_files/2004_metro_tax_parcels.txt')

In [70]:
addcolumns = pipeable(lambda chunk: (chunk 
                                             >> select(cols_to_keep)
                                             >> mutate(lat_long = pd.Series(zip(firstchunk.centroid_lat, firstchunk.centroid_long)).astype(str))
                                             >> mutate(lake_code = recode(X.lat_long, latlong_to_code).astype(str))
                                             >> mutate(lake_name = recode(X.lat_long, latlong_to_name).astype(str))
                                             >> mutate(parcel_dist_to_lake = recode(X.lat_long, latlong_to_distance).astype(str))
                                             >> drop(X.lat_long)
                                             >> filter_by(~X.lake_name.isna())
                                             >> filter_by(~X.parcel_dist_to_lake.astype(float).isna())
                                             >> filter_by(X.parcel_dist_to_lake.astype(float) <= 1600)
                            )     )    

In [71]:
addcolumns(firstchunk)

,ACRES_DEED,ACRES_POLY,AGPRE_ENRD,AG_PRESERV,BASEMENT,CITY,COOLING,DWELL_TYPE,EMV_BLDG,EMV_LAND,...,XUSE2_DESC,XUSE3_DESC,XUSE4_DESC,YEAR_BUILT,Year,centroid_lat,centroid_long,lake_code,lake_name,parcel_dist_to_lake


In [72]:
ch = first(fileiterator[4])
addcolumns(ch)

AttributeError: 'TextFileReader' object has no attribute 'iloc'

In [73]:
ch2 = next(fileiterator[5])
addcolumns(ch2)

/Users/tr8934qw/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ACRES_DEED,ACRES_POLY,AGPRE_ENRD,AG_PRESERV,BASEMENT,CITY,COOLING,DWELL_TYPE,EMV_BLDG,EMV_LAND,...,XUSE2_DESC,XUSE3_DESC,XUSE4_DESC,YEAR_BUILT,Year,centroid_lat,centroid_long,lake_code,lake_name,parcel_dist_to_lake


6. We need to make a unique primary key for each row in the combined parcel file.<br>
    a. There is a column that appears to be a unique parcel id.  Double check that this is a true primary key for each individual file. (To do this you need to verify that the number of unique values is the same as the number of rows for each of the parcel files.  **Hint:** For each file, use of the accumulator pattern with two accumualtors (one number and one data frame). <br>
    b. Explain why this column will not work as a primary key if we want to combine all years in one database. <br>
    c. Suppose we make a new column that consist of `str(year) + '-' + PIN`.  Explain why this should make a proper primary key for the combined data. <br>

In [22]:
firstchunks = [first(f) for f in fileiterator]

AttributeError: 'TextFileReader' object has no attribute 'iloc'

In [74]:
#list(enumerate(firstchunk.columns))
firstchunk.WSHD_DIST.unique()

array(['UPPER RUM RIVER WMO', 'SIX CITIES WMO',
       'RICE CREEK WATERSHED DISTRICT', 'COON CREEK WATERSHED DISTRICT',
       'LOWER RUM RIVER WMO', nan, 'SIX CITIES WATERSHED MGMT ORG',
       'SUNRISE RIVER WMO', 'VADNAIS LAKE AREA WMO'], dtype=object)

In [75]:
string_dtypes = {k:str for k in firstchunk.columns}

In [76]:
nrows = 0
unique_list = []
for ch in firstchunks:
    nrows = nrows + len(ch)
    print(nrows)
    for x in ch['PIN']:
        if x not in unique_list:
            unique_list.append(x)
    #print(sum(unique_list))    
    #pinset = set(ch['PIN'])
    #unique = unique 
    #unique = unique + ch['PIN'].intersection(unique)
    #unique = pinset.intersection(pinset)
    #print(unique)

NameError: name 'firstchunks' is not defined

In [135]:
#[(col, exp[col].is_unique) for col in exp]

This columm will not work as a primary key if we want to combine all the years because there isn't a difference between each year, we can't tell them apart.

str(year) + '-' + PIN should make a proper primary key because adding the year will make the key unique.

7. Make a function to add the key suggested in the last problem (`str(year) + '-' + PIN`) to a given chunk.

In [77]:
add_primary_key = pipeable(lambda start, df: (df
                                              >> mutate(id = np.arange(start, start + len(df))
                                              )))

In [78]:
c_size = 50000
from dfply import head
i = 0
(exp 
 >> add_primary_key(i*c_size) 
 >> head(3))

,ACRES_DEED,ACRES_POLY,AGPRE_ENRD,AG_PRESERV,BASEMENT,CITY,COOLING,DWELL_TYPE,EMV_BLDG,EMV_LAND,...,XUSE3_DESC,XUSE4_DESC,YEAR_BUILT,Year,centroid_lat,centroid_long,lake_code,lake_name,parcel_dist_to_lake,id


In [79]:
process_chunk = pipeable(lambda i, df, chunksize=c_size: df >> addcolumns >> add_primary_key(i*c_size))
exp >> process_chunk(0) >> tail(1)

,ACRES_DEED,ACRES_POLY,AGPRE_ENRD,AG_PRESERV,BASEMENT,CITY,COOLING,DWELL_TYPE,EMV_BLDG,EMV_LAND,...,XUSE3_DESC,XUSE4_DESC,YEAR_BUILT,Year,centroid_lat,centroid_long,lake_code,lake_name,parcel_dist_to_lake,id


In [80]:
from sqlalchemy import String, Float, Integer, DateTime
common_parcel_types = {'ACRES_DEED':Float,
                       'ACRES_POLY':Float,
                       'AGPRE_ENRD':DateTime,
                       'AGPRE_EXPD':DateTime,
                       'AG_PRESERV':String,
                       'BASEMENT':String,
                       'BLDG_NUM':Float,
                       'BLOCK':String,
                       'CITY':String,
                       'CITY_USPS':String,
                       'COOLING':String,
                       'COUNTY_ID':Integer,
                       'DWELL_TYPE':String,
                       'EMV_BLDG':Float,
                       'EMV_LAND':Float,
                       'EMV_TOTAL':Float,
                       'FIN_SQ_FT':Float,
                       'GARAGE':String,
                       'GARAGESQFT':Float,
                       'GREEN_ACRE':Float,
                       'HEATING':String,
                       'HOMESTEAD':String,
                       'HOME_STYLE':Float,
                       'ID':String,
                       'LANDMARK':String,
                       'LOT':String,
                       'MULTI_USES':Float,
                       'NUM_UNITS':String,
                       'OPEN_SPACE':Float,
                       'OWNER_MORE':String,
                       'OWNER_NAME':String,
                       'OWN_ADD_L1':String,
                       'OWN_ADD_L2':String,
                       'OWN_ADD_L3':String,
                       'PARC_CODE':Float,
                       'PIN':String,
                       'PLAT_NAME':String,
                       'PREFIXTYPE':Float, 
                       'PREFIX_DIR':String,
                       'SALE_DATE':Integer, 
                       'SALE_VALUE':Float,
                       'SCHOOL_DST':Float, 
                       'SPEC_ASSES':Float,
                       'STREETNAME':String,
                       'STREETTYPE':String, 
                       'SUFFIX_DIR':String, 
                       'Shape_Area':Float, 
                       'Shape_Leng':Float, 
                       'TAX_ADD_L1':String, 
                       'TAX_ADD_L2':String, 
                       'TAX_ADD_L3':String, 
                       'TAX_CAPAC':Float, 
                       'TAX_EXEMPT':String, 
                       'TAX_NAME':String, 
                       'TOTAL_TAX':Float, 
                       'UNIT_INFO':String, 
                       'USE1_DESC':String,
                       'USE2_DESC':String,
                       'USE3_DESC':String,
                       'USE4_DESC':String,
                       'WSHD_DIST':String,
                       'XUSE1_DESC':String,
                       'XUSE2_DESC':String,
                       'XUSE3_DESC':String,
                       'XUSE4_DESC':String,
                       'YEAR_BUILT':Integer,
                       'Year':String,
                       'ZIP':Integer, 
                       'ZIP4':Integer,
                       'centroid_lat':String,
                       'centroid_long':String}

In [81]:
from more_sqlalchemy import get_sql_types
i = 0
complete_first_chunk = exp >> process_chunk(i)
sql_types = common_parcel_types1
#sql_types = get_sql_types(complete_first_chunk)
sql_types

{'ACRES_DEED': sqlalchemy.sql.sqltypes.String,
 'ACRES_POLY': sqlalchemy.sql.sqltypes.String,
 'AGPRE_ENRD': sqlalchemy.sql.sqltypes.String,
 'AGPRE_EXPD': sqlalchemy.sql.sqltypes.String,
 'AG_PRESERV': sqlalchemy.sql.sqltypes.String,
 'BASEMENT': sqlalchemy.sql.sqltypes.String,
 'BLDG_NUM': sqlalchemy.sql.sqltypes.String,
 'BLOCK': sqlalchemy.sql.sqltypes.String,
 'CITY': sqlalchemy.sql.sqltypes.String,
 'CITY_USPS': sqlalchemy.sql.sqltypes.String,
 'COOLING': sqlalchemy.sql.sqltypes.String,
 'COUNTY_ID': sqlalchemy.sql.sqltypes.String,
 'DWELL_TYPE': sqlalchemy.sql.sqltypes.String,
 'EMV_BLDG': sqlalchemy.sql.sqltypes.String,
 'EMV_LAND': sqlalchemy.sql.sqltypes.String,
 'EMV_TOTAL': sqlalchemy.sql.sqltypes.String,
 'FIN_SQ_FT': sqlalchemy.sql.sqltypes.String,
 'GARAGE': sqlalchemy.sql.sqltypes.String,
 'GARAGESQFT': sqlalchemy.sql.sqltypes.String,
 'GREEN_ACRE': sqlalchemy.sql.sqltypes.String,
 'HEATING': sqlalchemy.sql.sqltypes.String,
 'HOMESTEAD': sqlalchemy.sql.sqltypes.String,


In [82]:
from sqlalchemy import String, Float, Integer, DateTime
common_parcel_types1 = {'ACRES_DEED':String,
                       'ACRES_POLY':String,
                       'AGPRE_ENRD':String,
                       'AGPRE_EXPD':String,
                       'AG_PRESERV':String,
                       'BASEMENT':String,
                       'BLDG_NUM':String,
                       'BLOCK':String,
                       'CITY':String,
                       'CITY_USPS':String,
                       'COOLING':String,
                       'COUNTY_ID':String,
                       'DWELL_TYPE':String,
                       'EMV_BLDG':String,
                       'EMV_LAND':String,
                       'EMV_TOTAL':String,
                       'FIN_SQ_FT':String,
                       'GARAGE':String,
                       'GARAGESQFT':String,
                       'GREEN_ACRE':String,
                       'HEATING':String,
                       'HOMESTEAD':String,
                       'HOME_STYLE':String,
                       'ID':Integer,
                       'LANDMARK':String,
                       'LOT':String,
                       'MULTI_USES':String,
                       'NUM_UNITS':String,
                       'OPEN_SPACE':String,
                       'OWNER_MORE':String,
                       'OWNER_NAME':String,
                       'OWN_ADD_L1':String,
                       'OWN_ADD_L2':String,
                       'OWN_ADD_L3':String,
                       'PARC_CODE':String,
                       'PIN':String,
                       'PLAT_NAME':String,
                       'PREFIXTYPE':String,
                       'PREFIX_DIR':String,
                       'SALE_DATE':String,
                       'SALE_VALUE':String,
                       'SCHOOL_DST':String,
                       'SPEC_ASSES':String,
                       'STREETNAME':String,
                       'STREETTYPE':String,
                       'SUFFIX_DIR':String,
                       'Shape_Area':String,
                       'Shape_Leng':String,
                       'TAX_ADD_L1':String,
                       'TAX_ADD_L2':String,
                       'TAX_ADD_L3':String,
                       'TAX_CAPAC':String,
                       'TAX_EXEMPT':String,
                       'TAX_NAME':String,
                       'TOTAL_TAX':String,
                       'UNIT_INFO':String,
                       'USE1_DESC':String,
                       'USE2_DESC':String,
                       'USE3_DESC':String,
                       'USE4_DESC':String,
                       'WSHD_DIST':String,
                       'XUSE1_DESC':String,
                       'XUSE2_DESC':String,
                       'XUSE3_DESC':String,
                       'XUSE4_DESC':String,
                       'YEAR_BUILT':String,
                       'Year':String,
                       'ZIP':String,
                       'ZIP4':String,
                       'centroid_lat':String,
                       'centroid_long':String}

In [83]:
!rm ./databases/lakes.db

In [84]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///databases/lakes.db', echo=False)

In [85]:
schema = pd.io.sql.get_schema(complete_first_chunk, # dataframe
                              'lakes', # name in SQL db
                              keys='id', # primary key
                              con=engine, # connection
                              dtype=sql_types # SQL types
)
print(schema)
engine.execute(schema)


CREATE TABLE lakes (
	"ACRES_DEED" VARCHAR, 
	"ACRES_POLY" VARCHAR, 
	"AGPRE_ENRD" VARCHAR, 
	"AG_PRESERV" VARCHAR, 
	"BASEMENT" VARCHAR, 
	"CITY" VARCHAR, 
	"COOLING" VARCHAR, 
	"DWELL_TYPE" VARCHAR, 
	"EMV_BLDG" VARCHAR, 
	"EMV_LAND" VARCHAR, 
	"FIN_SQ_FT" VARCHAR, 
	"GARAGE" VARCHAR, 
	"GARAGESQFT" VARCHAR, 
	"GREEN_ACRE" VARCHAR, 
	"HOMESTEAD" VARCHAR, 
	"LANDMARK" VARCHAR, 
	"OWN_ADD_L1" VARCHAR, 
	"OWN_ADD_L2" VARCHAR, 
	"OWN_ADD_L3" VARCHAR, 
	"PARC_CODE" VARCHAR, 
	"PIN" VARCHAR, 
	"SALE_VALUE" VARCHAR, 
	"SPEC_ASSES" VARCHAR, 
	"TAX_CAPAC" VARCHAR, 
	"TAX_EXEMPT" VARCHAR, 
	"TOTAL_TAX" VARCHAR, 
	"USE1_DESC" VARCHAR, 
	"USE2_DESC" VARCHAR, 
	"USE3_DESC" VARCHAR, 
	"USE4_DESC" VARCHAR, 
	"WSHD_DIST" VARCHAR, 
	"XUSE1_DESC" VARCHAR, 
	"XUSE2_DESC" VARCHAR, 
	"XUSE3_DESC" VARCHAR, 
	"XUSE4_DESC" VARCHAR, 
	"YEAR_BUILT" VARCHAR, 
	"Year" VARCHAR, 
	centroid_lat VARCHAR, 
	centroid_long VARCHAR, 
	lake_code TEXT, 
	lake_name TEXT, 
	parcel_dist_to_lake TEXT, 
	id BIGINT NOT NULL, 

In [86]:
rows_so_far = 0
nrows = rows_so_far
for f in files:
    c_size = 50000
    print('processing file {0}'.format(f))
    df_iter = enumerate(pd.read_csv(f, 
                                chunksize = c_size, sep = '|', 
                                    dtype=string_dtypes))
    for i, chunk in df_iter:
        processed_chunk = chunk >> process_chunk(nrows)
        print('writing chunk {0}'.format(i))
        nrows = nrows + len(chunk)
        processed_chunk.to_sql('lakes', 
                           con=engine, 
                           dtype=sql_types, 
                           index=False,
                           if_exists='append')

processing file ./data/MinneMUDAC_raw_files/2004_metro_tax_parcels.txt
writing chunk 0
writing chunk 1
writing chunk 2
writing chunk 3
writing chunk 4
writing chunk 5
writing chunk 6
writing chunk 7
writing chunk 8
writing chunk 9
writing chunk 10
writing chunk 11
writing chunk 12
writing chunk 13
writing chunk 14
writing chunk 15
writing chunk 16
writing chunk 17
writing chunk 18
writing chunk 19
writing chunk 20
writing chunk 21
writing chunk 22
writing chunk 23
writing chunk 24
writing chunk 25
writing chunk 26
writing chunk 27
writing chunk 28
writing chunk 29
writing chunk 30
writing chunk 31
writing chunk 32
writing chunk 33
writing chunk 34
writing chunk 35
writing chunk 36
writing chunk 37
writing chunk 38
writing chunk 39
processing file ./data/MinneMUDAC_raw_files/2005_metro_tax_parcels.txt
writing chunk 0
writing chunk 1
writing chunk 2
writing chunk 3
writing chunk 4
writing chunk 5
writing chunk 6
writing chunk 7
writing chunk 8
writing chunk 9
writing chunk 10
writing chu

/Users/tr8934qw/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (20,24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


writing chunk 23


/Users/tr8934qw/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (20,24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


writing chunk 24
writing chunk 25
writing chunk 26
writing chunk 27
writing chunk 28
writing chunk 29
writing chunk 30
writing chunk 31
writing chunk 32
writing chunk 33
writing chunk 34
writing chunk 35
writing chunk 36
writing chunk 37
writing chunk 38
writing chunk 39
writing chunk 40
processing file ./data/MinneMUDAC_raw_files/2008_metro_tax_parcels.txt
writing chunk 0
writing chunk 1
writing chunk 2
writing chunk 3
writing chunk 4
writing chunk 5
writing chunk 6
writing chunk 7
writing chunk 8
writing chunk 9
writing chunk 10
writing chunk 11
writing chunk 12
writing chunk 13
writing chunk 14
writing chunk 15
writing chunk 16
writing chunk 17
writing chunk 18
writing chunk 19
writing chunk 20
writing chunk 21
writing chunk 22
writing chunk 23


...

writing chunk 24
writing chunk 25
writing chunk 26
writing chunk 27
writing chunk 28
writing chunk 29
writing chunk 30
writing chunk 31
writing chunk 32
writing chunk 33
writing chunk 34
writing chunk 35
writing chunk 36
writing chunk 37
writing chunk 38
writing chunk 39
writing chunk 40
writing chunk 41
processing file ./data/MinneMUDAC_raw_files/2011_metro_tax_parcels.txt
writing chunk 0
writing chunk 1
writing chunk 2
writing chunk 3
writing chunk 4
writing chunk 5
writing chunk 6
writing chunk 7
writing chunk 8
writing chunk 9
writing chunk 10
writing chunk 11
writing chunk 12
writing chunk 13
writing chunk 14
writing chunk 15
writing chunk 16
writing chunk 17
writing chunk 18
writing chunk 19
writing chunk 20
writing chunk 21
writing chunk 22
writing chunk 23
writing chunk 24
writing chunk 25
writing chunk 26
writing chunk 27
writing chunk 28
writing chunk 29
writing chunk 30
writing chunk 31
writing chunk 32
writing chunk 33
writing chunk 34
writing chunk 35
writing chunk 36
writ

/Users/tr8934qw/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (56) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


writing chunk 27
writing chunk 28
writing chunk 29
writing chunk 30
writing chunk 31
writing chunk 32
writing chunk 33
writing chunk 34
writing chunk 35


/Users/tr8934qw/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (56) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


writing chunk 36
writing chunk 37
writing chunk 38
writing chunk 39
writing chunk 40
writing chunk 41
writing chunk 42


#### Note: If you are clever, you can do parts 8 in one double loop, which will save you from having to read the parcel files twice.

8. It is probably worth our time to test that our new key column is truely unique. (If not, we might be wasting out time loading the data into a database, only to have process fail hours in.) Test that the new column works by <br>
    a. Iterating over all the files.<br>
    b. Using an accumulator to count total number of rows across all parcel files. <br>
    c. Using an accumulator to accumulate a set of all unique values of our new key. <br>
    d. Verifying that we have as many total rows as unique keys.
    a. Selecting just this column. <br>
    b. Dumping this column into a temporary database <br>

9. If the last step succeeded, you can proceed to make a master parcel data database.  If not, you will need to figure out another primary key, probably an `id` column similar to the example in the lectures.